In [ ]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn import utils
from sklearn.metrics import accuracy_score


In [ ]:
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')

print(movies.head())
print(ratings.head())

In [ ]:
moviesdb = ratings.pivot(index = 'movieId', columns = 'userId', values = 'rating')

moviesdb.head()

In [ ]:
moviesdb.fillna(0, inplace=True)

moviesdb.head()

In [ ]:
nouser = ratings.groupby('movieId')['rating'].agg('count')

nomovies = ratings.groupby('userId')['rating'].agg('count')


In [ ]:
moviesdb = moviesdb.loc[:, nomovies[nomovies > 50].index]

moviesdb

In [ ]:
moviesdb = moviesdb.loc[nouser[nouser > 10].index]

moviesdb

In [ ]:
csr_data = csr_matrix(moviesdb.values)

moviesdb.reset_index(inplace = True)

moviesdb

In [ ]:
knn = NearestNeighbors(metric= 'cosine', algorithm= 'brute', n_neighbors= 20, n_jobs= -1)

knn.fit(csr_data)

In [ ]:
def getrecs(movie):
    moviestorec = 10
    movielist = movies[movies['title'].str.contains(movie)]
    if len(movielist):
        movieindex = movielist.iloc[0]['movieId']
        movieindex = moviesdb[moviesdb['movieId'] == movieindex].index[0]
        distances, indices = knn.kneighbors(csr_data[movieindex], n_neighbors = moviestorec+1)
        recmovie = sorted(list(zip(indices.squeeze().tolist(), distances.squeeze().tolist())), key = lambda x: x[1])[:0:-1]
        recframe = []

        for val in recmovie:
            movieindex = moviesdb.iloc[val[0]]['movieId']
            idex = movies[movies['movieId'] == movieindex].index
            recframe.append({'Title':movies.iloc[idex]['title'].values[0], 'Distance': val[1]})
        df = pd.DataFrame(recframe, index = range(1, moviestorec+1))
        return df
    else: 
        return 'You get nothing you lose. Good day Sir!'
    


In [ ]:
getrecs('Iron Man')

In [ ]:
getrecs('Wizard of Oz')

In [ ]:
# Separate the y variable, the labels
y = ratings['rating']

# Separate the X variable, the features
x = ratings.drop(columns= ['rating'])

In [ ]:
lab = preprocessing.LabelEncoder()
y_trans = lab.fit_transform(y)
y_trans

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x,y_trans,test_size=0.25,random_state=42)

In [ ]:
x=x.transpose()

In [ ]:
x.shape

In [ ]:
y_trans.shape

In [ ]:
knn = KNeighborsClassifier(n_neighbors=10)
knn.fit(x_train,y_train)

In [ ]:
pre = knn.predict(x_test)

In [ ]:
accuracy_score(pre,y_test)